In [ ]:
import lightning.pytorch as pl
from wav2vec2 import RequestsDataset, EmotionDataset, AudioModel
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from bert import TranscriptData, Classifier
from sklearn.metrics import recall_score

tqdm.pandas()
train_df = pd.read_csv('HCC/lab/train.csv')
develdf = pd.read_csv('HCC/lab/devel.csv')
testdf = pd.read_csv('HCC/lab/test.csv')

In [ ]:

le = RequestsDataset.get_le(train_df,target='request')
devel = RequestsDataset(develdf, data_path='HCC/wav/',le=le,max_sec=30,test=True,target='request')
test = RequestsDataset(testdf, data_path='HCC/wav/',max_sec=30, test=True,target='request')

In [ ]:
ckpt_path = 'req_ckpts/wav2vec2-com.ckpt'
model = AudioModel(2)
devel_loader = torch.utils.data.DataLoader(devel, batch_size=16, shuffle=False)
test_loader = torch.utils.data.DataLoader(test, batch_size=16, shuffle=False)

trainer = pl.Trainer(
    devices=1,
    accelerator='gpu'
)

In [ ]:
y = devel.labels

preds = trainer.predict(model,devel_loader,ckpt_path=ckpt_path)
y_hat = torch.cat(preds).detach().cpu().numpy()
print(recall_score(y,y_hat.argmax(axis=-1),average='macro'))

In [ ]:
test_preds = trainer.predict(model,test_loader,ckpt_path=ckpt_path)
test_preds = torch.cat(test_preds).detach().cpu().numpy()
testdf['request'] = le.inverse_transform(test_preds.argmax(axis=-1))

In [ ]:
testdf.to_csv('intermediate.csv',index=False)
bert_model = Classifier.load_from_checkpoint('distilbert_com/distil-com.ckpt')
isinstance(bert_model,pl.LightningModule)

In [ ]:

traindata = TranscriptData(train_df, transcript_path='HCC/transcripts',target='complaint')
devdata = TranscriptData(develdf, transcript_path='HCC/transcripts',test=True,target='complaint',le=traindata.le)
testdata = TranscriptData(testdf, transcript_path='HCC/transcripts',test=True,target='complaint')

In [ ]:
devel_loader = torch.utils.data.DataLoader(devdata, batch_size=16, shuffle=False)
test_loader = torch.utils.data.DataLoader(testdata, batch_size=16, shuffle=False)

bert_model = bert_model.to('cuda:0')
preds = []
for batch in tqdm(devel_loader):
    x0, x1= batch
    x0 = {k:v.to('cuda:0') for k,v in x0.items()}
    x1 = {k:v.to('cuda:0') for k,v in x1.items()}
    y_hat = bert_model(x0,x1)
    preds.append(y_hat.detach().cpu().numpy())

In [ ]:
labels = devdata.df['label'].values
y_hat = np.concatenate(preds).argmax(axis=-1)
print(recall_score(labels,y_hat,average='macro'))

In [ ]:
preds = []
for batch in tqdm(test_loader):
    x0, x1= batch
    x0 = {k:v.to('cuda:0') for k,v in x0.items()}
    x1 = {k:v.to('cuda:0') for k,v in x1.items()}
    y_hat = bert_model(x0,x1)
    preds.append(y_hat.detach().cpu().numpy())

In [ ]:
testdf = pd.read_csv('intermediate.csv')
y_hat = np.concatenate(preds).argmax(axis=-1)
testdf['complaint'] = traindata.le.inverse_transform(y_hat)
submission = testdf[['filename','complaint','request']]
submission.to_csv('HCC.csv',index=False)
submission.head()

In [ ]:
develdf = pd.read_csv('emotions/lab/devel.csv')
testdf = pd.read_csv('emotions/lab/test.csv')
dev_emotions = EmotionDataset(develdf, data_path='emotions/raw/wav/', test=True)
test_emotions = EmotionDataset(testdf, data_path='emotions/raw/wav/', test=True)

ckpt_path = 'ckpts/epoch=6-step=9261.ckpt'
model = AudioModel(9,task='emotions')
devel_loader = torch.utils.data.DataLoader(dev_emotions, batch_size=16, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_emotions, batch_size=16, shuffle=False)
trainer = pl.Trainer(
    devices=1,
    accelerator='gpu'
)

In [ ]:
preds = trainer.predict(model,test_loader,ckpt_path=ckpt_path)

In [ ]:
y_hat = torch.sigmoid(torch.cat(preds)).detach().cpu().numpy()
print(y_hat.shape)
submission = testdf[['filename']].copy()
labels = list(testdf.columns[1:])
for i in range(len(labels)):
    submission[labels[i]] = y_hat[:,i].reshape(-1)
submission.head()

In [ ]:
submission.to_csv('emotions.csv',index=False)